In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from imgaug import augmenters as iaa
import torch
from tqdm import tqdm_notebook as tqdm

torch.backends.cudnn.benchmark = True

from dataloader import get_test_dataloader
from preprocess import transformer, img_transform, reverse_img_transform
from models import RetinaNet
from train import load_components
from inference import InferenceTransform
from utils import Visualiser
import pickle
import gc
import json
from evaluation import support_evaluate_model
from collections import defaultdict
gc.collect(2)

/opt/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


0

In [3]:
def perform_inference(test_dl, config_dir, list_results_dict):
    parameters = json.load(open(config_dir,'r'))

    hyper_params = parameters["hyperparams"]
    dir_params = parameters["dirparams"]
    project_params = parameters["projectconfig"]

    clsids_to_names = json.load(open(dir_params["clsids_to_names_dir"],'r'))
    clsids_to_idx = json.load(open(dir_params["clsids_to_idx_dir"],'r'))
    idx_to_cls_ids = {v: k for k, v in clsids_to_idx.items()}
    idx_to_names = {k: clsids_to_names[v] for k, v in idx_to_cls_ids.items()}

    retinanet = RetinaNet(
            hyper_params["backbone"], 
            hyper_params["num_classes"],
            hyper_params["ratios"], 
            hyper_params["scales"], 
            device=hyper_params["device"], 
            pretrained = hyper_params["pretrained"], 
            freeze_bn = hyper_params["freeze_bn"],
            prior=0.01, 
            feature_size=256, 
            pyramid_levels = [3, 4, 5, 6, 7],
            criterion=None
        )
    retinanet = retinanet.to(hyper_params["device"])

    vis = Visualiser(
            hyper_params["num_classes"],
            idx_to_names,
            reverse_img_transform
        )

    inferencer = InferenceTransform(
            idx_to_names,
            idx_to_cls_ids,
            hyper_params["regress_factor"]
        ) 

    load_components(retinanet, None, None, hyper_params["save_dir"]+"/final.pth")

    if torch.cuda.device_count() > 1:
        print("Using Multiple GPUs")
        retinanet = torch.nn.DataParallel(retinanet, device_ids=range(torch.cuda.device_count())) 
    retinanet = retinanet.to(hyper_params["device"])

    eval_params = {
        "overlap":hyper_params["overlap"],
        "cls_thresh": 0.05
    }
    
    resultant_dict = support_evaluate_model(retinanet, 
       test_dl, 
       inferencer, 
       vis, 
       cls_thresh=eval_params["cls_thresh"], 
       overlap=eval_params["overlap"], 
       device=hyper_params["device"], 
       save_dir="test", 
       display = False, 
       create_result=True,
       list_results_dict = list_results_dict)
    
    return resultant_dict

In [4]:
test_seq = iaa.Sequential([
        iaa.Resize({"height": 640, "width": 640})
    ])
test_transform_fn = transformer(test_seq, img_transform)
test_dl = get_test_dataloader(
        "dataset/test/test", 
        "data_info/test/test-idx_to_id.pkl", 
        test_transform_fn, 
        16, 
        4
    )

In [5]:
# configs_dir = [
#     "configs/final/subset0/1/config.json",
#     "configs/final/subset1/0/config.json",
#     "configs/final/subset2/0/config.json",
#     "configs/final/subset3/3/config.json",
#     "configs/final/subset4/5/config.json",
#     "configs/final/subset5/4/config.json",
# ]
configs_dir = [
    "configs/final/subset5/4/config.json",
    "configs/final/subset0/2/config.json",
    "configs/final/subset1/1/config.json",
    "configs/final/subset2/1/config.json",
    "configs/final/subset3/5/config.json",
    "configs/final/subset4/7/config.json",
]
out_file="submission2.csv"

In [6]:
list_results_dict = defaultdict(list)
for config_dir in configs_dir:
    list_results_dict = perform_inference(test_dl, config_dir, list_results_dict)

Loading from checkpoint: configs/final/subset5/4/final.pth
Using Multiple GPUs



Loading from checkpoint: configs/final/subset0/2/final.pth
Using Multiple GPUs



Loading from checkpoint: configs/final/subset1/1/final.pth
Using Multiple GPUs



Loading from checkpoint: configs/final/subset2/1/final.pth
Using Multiple GPUs



Loading from checkpoint: configs/final/subset3/5/final.pth
Using Multiple GPUs



Loading from checkpoint: configs/final/subset4/7/final.pth
Using Multiple GPUs


In [7]:
f = open(out_file,"w+")
results="ImageId,PredictionString\n"
f.write(results)
for key, value in tqdm(list_results_dict.items()):
    results=key+","
    for val in value:
        clsId = val[0]
        score = round(val[1],2)
        bbox = ' '.join(str(round(e,5)) for e in val[2])
        results+= str(clsId) + " "+ str(score)+ " " + bbox + " "
    results +="\n"
    f.write(results)
f.close()